# RAG

### IMPORTS AND INSTALLS

#### INSTALL LLAMA and the other utilties
`If there is an error in the installs, restart the runtime`

In [20]:
#@title installs
# TODO: I Could install this into the conda env
#%pip install llama-index-readers-file pymupdf
%pip install llama-index-vector-stores-postgres
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-llama-cpp


# RESTART RUNTIME after installing the above packages or if there is an error in the installation

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached llama_cpp_python-0.2.90-cp38-cp38-linux_x86_64.whl
  Attempting uninstall: llama-cpp-python
    Found existing installation: llama_cpp_python 0.2.24
    Uninstalling llama_cpp_python-0.2.24:
      Successfully uninstalled llama_cpp_python-0.2.24
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement llama-index-node_parser-html (from versions: none)
ERROR: No matching distribution found for llama-index-node_parser-html
Note: you may need to restart the kernel to use updated packages.


In [23]:
%pip install llama-index

  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.4 MB/s eta 0:00:00a 0:00:01
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.3/176.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 6.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 7.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 8.2 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
# using conda environment
%conda install -c conda-forge llama-cpp-python 
# Using pip
#%pip install llama-cpp-python

done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 24.5.0
  latest version: 24.9.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.9.0



## Package Plan ##

  environment location: /home/vicentamen/miniconda3/envs/py38

  added / updated specs:
    - llama-cpp-python


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    anyio-4.3.0                |     pyhd8ed1ab_0         100 KB  conda-forge
    cuda-version-12.4          |       h3060b56_3          21 KB  conda-forge
    exceptiongroup-1.2.2       |     pyhd8ed1ab_0          20 KB  conda-forge
    h2-4.1.0                   |   py38h578d9bd_0          77 KB  conda-forge
    llama-cpp-python-0.2.24    |   py38h17151c0_0         211 KB  conda-forge
    markups

#### Setup Embedings model
*The embedings model will work on the Indexing side of the RAG to create the embeddings of the HTML nodes*

In [18]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

No sentence-transformers model found with name BAAI/bge-small-en. Creating a new one with mean pooling.


#### Set up LLM Model
*The LLM will be in charge of the generation module and the pre-reasoning of the problem*

In [19]:
from llama_index.llms.llama_cpp import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

/home/vicentamen/miniconda3/envs/py38/lib/python3.8/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/vicentamen/miniconda3/envs/py38/lib/python3.8/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_path" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/vicentamen/miniconda3/envs/py38/lib/python3.8/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_kwargs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


total size (MB): 7365.83


7025it [26:05,  4.49it/s]                            
llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /tmp/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loa

#### Initialize postgress

In [32]:
# Start and enable PostgreSQL service
!service postgresql start
!service postgresql status

# What happens if the user does not have the required permissions to start the service?

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


● postgresql.service - PostgreSQL RDBMS
     Loaded: loaded (]8;;file://ventagram/lib/systemd/system/postgresql.service/lib/systemd/system/postgresql.service]8;;; enabled; preset: enabled)
     Active: active (exited) since Wed 2024-10-02 09:41:28 CEST; 3h 50min ago
   Main PID: 2128 (code=exited, status=0/SUCCESS)
        CPU: 1ms

Oct 02 09:41:28 ventagram systemd[1]: Starting postgresql.service - Postgre…S...
Oct 02 09:41:28 ventagram systemd[1]: Finished postgresql.service - Postgre…BMS.
Hint: Some lines were ellipsized, use -l to show in full.


In [20]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

In [21]:
"""
conn = psycopg2.connect(
    host="localhost",
    port=5432,
    database="rag_db",
    user="rag_user",
    password="1234"
)
"""

'\nconn = psycopg2.connect(\n    host="localhost",\n    port=5432,\n    database="rag_db",\n    user="rag_user",\n    password="1234"\n)\n'

In [46]:
import psycopg2

db_name = "rag_db"
host = "localhost"
password = "1234"
port = "5432"
user = "rag_user"
# conn = psycopg2.connect(connection_string)

conn = psycopg2.connect(
    dbname=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

""" with conn.cursor() as c:
    c.execute(f"DROP DATABASE IF EXISTS {db_name}")
    c.execute(f"CREATE DATABASE {db_name}") """


' with conn.cursor() as c:\n    c.execute(f"DROP DATABASE IF EXISTS {db_name}")\n    c.execute(f"CREATE DATABASE {db_name}") '

In [37]:
from sqlalchemy import make_url
from llama_index.vector_stores.postgres import PGVectorStore

vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
    table_name="llama2_paper",
    embed_dim=384,  # openai embedding dimension
)

## INDEXING

### LOAD

In [39]:
#@title Import Mind2Web dataset form huggingface
from datasets import load_dataset

# Text only dataset
ds = load_dataset("osunlp/Mind2Web")
# Multimodal dataset
#ds = load_dataset("osunlp/Multimodal-Mind2Web")

In [40]:
# Get the train split
train = ds["train"]

In [41]:
# Get an example task from the dataset for testing
task = train[10]

# Extract the cleant_html from each action of the task and transform into llama-index document
task_html = task["actions"][0]["cleaned_html"]
print(len(task_html))

292852


In [49]:
task_prompt = task["confirmed_task"]

KeyError: 'prompt'

### SPLITTING

In [42]:
from llama_index.core.node_parser import HTMLNodeParser
from llama_index.core import Document

# Create a node parser
node_parser = HTMLNodeParser()

# Parse the the HTMLs from the task
nodes = node_parser.get_nodes_from_documents([Document(text=task_html)], show_progress=True)
# NOTE: We can preprocess the nodes before indexing them to save time

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Parsing nodes: 100%|██████████| 1/1 [00:00<00:00,  2.80it/s]


### EMBEDDING

In [47]:
# Generate embeddings for each node
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

In [48]:
vector_store.add(nodes)

['a8f864a8-07f0-4af4-9a27-3ba2cc388673',
 'cbf8f571-995c-42f5-8d48-a99a372e4e9a',
 '8b33ae7e-8816-4196-beff-4f1a4ddd4af5',
 '9f211c32-e10a-4abe-866a-6f2d26dcb82f',
 'b5f8c7d0-d894-4fe2-9189-e64ac23e6e3d',
 '4f652d9b-948e-42b2-9435-9734ce9a5adb',
 '56017ce3-0d6f-42c3-9737-954eb9f27115',
 '50987e4d-19f2-4ccc-ad0b-f6f063d06912',
 '3972e994-7ebf-4789-b88c-d0c76eae8f76',
 'e6295461-941a-40ab-8ccf-4a00c2112dcd',
 'b6057d2f-344d-451c-90c3-7b72fba01e48',
 'd1d543a9-95c3-40c7-b632-c49a5ff8abf5',
 '826b42e1-c566-4d73-8dda-867a9bd551b4',
 '56ef51cd-4e64-4c53-8c1f-b38cc34f55a3',
 '795bb31e-3605-481f-be27-97a0ad479635',
 'ffb2c0d8-56d8-4c6f-b63d-08dbad6863c6',
 '1d900f3b-2702-4433-b834-aaa788f1f83a',
 'b6565ae6-d78c-4563-8860-602bc34c6775',
 'f5185cf8-e7ed-4371-82bb-a8a7527abdd4',
 '4949646e-a024-4250-9fd2-8c6eb5f8eb8a',
 '24d93396-a71b-431b-b531-2cd1f1c73e69',
 'b335a6f4-382d-494f-ab50-bc341a1d2965',
 '45169495-a70b-4965-b907-9a44a42a2f85',
 'eff14371-657e-4248-84ac-346576a69080',
 '1e63364a-b55e-

## RETRIEVAL

In [ ]:
query_str = "Can you tell me about the key concepts for safety finetuning"

In [ ]:
#query_embedding = embed_model.get_query_embedding(query_str)
query_embedding = embed_model.get_query_embedding(task_prompt)

In [ ]:
# construct vector store query
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

## GENERATION